In [ ]:
# %% Cell 1: Setup
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from src.config import get_config
from src.data.data_loader import DataLoader
from src.features.time_domain import TimeDomainFeatures
from src.features.feature_engineering import FeatureEngineer

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ Imports successful")

# %% Cell 2: Load Data
config = get_config()
loader = DataLoader(config)

print("Loading machine data...")
df = loader.load_machine_data("machine_001")

# Use subset for faster processing
df_sample = df.iloc[:100000].copy()

print(f"Data shape: {df_sample.shape}")
print(f"Columns: {df_sample.columns.tolist()}")

# %% Cell 3: Explore Raw Signal
print("\n" + "="*80)
print("RAW SIGNAL ANALYSIS")
print("="*80)

# Take a 1-second window (100 samples at 100Hz)
window = df_sample['vibration_rms'].iloc[1000:1100].values

print(f"\nSignal statistics:")
print(f"  Mean: {np.mean(window):.4f}")
print(f"  Std: {np.std(window):.4f}")
print(f"  Min: {np.min(window):.4f}")
print(f"  Max: {np.max(window):.4f}")
print(f"  RMS: {np.sqrt(np.mean(window**2)):.4f}")

# Plot signal
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

# Time domain
axes[0].plot(window, linewidth=1)
axes[0].set_title('Vibration Signal (1 second)', fontweight='bold')
axes[0].set_xlabel('Sample')
axes[0].set_ylabel('Amplitude')
axes[0].grid(True, alpha=0.3)

# Histogram
axes[1].hist(window, bins=20, edgecolor='black', alpha=0.7)
axes[1].set_title('Amplitude Distribution', fontweight='bold')
axes[1].set_xlabel('Amplitude')
axes[1].set_ylabel('Frequency')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../results/figures/raw_signal_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

# %% Cell 4: Extract Features from Single Window
print("\n" + "="*80)
print("SINGLE WINDOW FEATURE EXTRACTION")
print("="*80)

extractor = TimeDomainFeatures()

# Extract features
features = extractor.extract_all_features(window, prefix='vib_')

print(f"\nExtracted {len(features)} features:")
print("\nBasic Statistics:")
for key in ['vib_mean', 'vib_std', 'vib_min', 'vib_max', 'vib_median']:
    print(f"  {key}: {features[key]:.4f}")

print("\nStatistical Moments:")
for key in ['vib_skewness', 'vib_kurtosis']:
    print(f"  {key}: {features[key]:.4f}")

print("\nSignal Characteristics:")
for key in ['vib_rms', 'vib_crest_factor', 'vib_peak_to_peak']:
    print(f"  {key}: {features[key]:.4f}")

print("\nShape Factors:")
for key in ['vib_shape_factor', 'vib_impulse_factor', 'vib_clearance_factor']:
    print(f"  {key}: {features[key]:.4f}")

# %% Cell 5: Feature Extraction Pipeline
print("\n" + "="*80)
print("FULL FEATURE ENGINEERING PIPELINE")
print("="*80)

engineer = FeatureEngineer(config)

print("Extracting features...")
print("  Window size: 1000 samples (10 seconds)")
print("  Stride: 500 samples (5 seconds)")

features_df = engineer.engineer_features(
    df_sample,
    window_size=1000,
    stride=500,
    include_rolling=True
)

print(f"\n✓ Feature extraction complete!")
print(f"  Feature windows: {len(features_df):,}")
print(f"  Features per window: {len(features_df.columns)}")
print(f"  Memory usage: {features_df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# %% Cell 6: Feature Overview
print("\n" + "="*80)
print("FEATURE OVERVIEW")
print("="*80)

print(f"\nFirst few rows:")
print(features_df.head())

print(f"\nFeature types:")
feature_cols = [col for col in features_df.columns 
               if col not in ['window_id', 'window_start', 'window_end', 'machine_id', 
                            'is_anomaly', 'failure_type', 'severity']]

# Count features by type
time_domain = [col for col in feature_cols if any(x in col for x in ['mean', 'std', 'rms', 'kurtosis', 'skewness', 'peak', 'crest'])]
rolling = [col for col in feature_cols if 'rolling' in col or 'ewma' in col]
roc = [col for col in feature_cols if 'roc' in col or 'pct_change' in col or 'momentum' in col]

print(f"  Time-domain features: {len(time_domain)}")
print(f"  Rolling window features: {len(rolling)}")
print(f"  Rate of change features: {len(roc)}")
print(f"  Total: {len(feature_cols)}")

# %% Cell 7: Feature Statistics
print("\n" + "="*80)
print("FEATURE STATISTICS")
print("="*80)

stats_df = engineer.get_feature_importance_analysis(features_df)

print(f"\nTop 15 Features by Variance:")
print(stats_df.head(15)[['feature', 'mean', 'std', 'variance', 'cv']])

# Plot top features by variance
fig, ax = plt.subplots(figsize=(12, 6))

top_features = stats_df.head(20)
ax.barh(range(len(top_features)), top_features['variance'], color='skyblue', edgecolor='black')
ax.set_yticks(range(len(top_features)))
ax.set_yticklabels([f.replace('vibration_rms_', 'vib_') for f in top_features['feature']], fontsize=8)
ax.set_xlabel('Variance', fontsize=10, fontweight='bold')
ax.set_title('Top 20 Features by Variance', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.savefig('../results/figures/feature_variance.png', dpi=300, bbox_inches='tight')
plt.show()

# %% Cell 8: Feature Correlation
print("\n" + "="*80)
print("FEATURE CORRELATION ANALYSIS")
print("="*80)

# Select subset of features for correlation
selected_features = [
    'vibration_rms_mean', 'vibration_rms_std', 'vibration_rms_rms',
    'vibration_rms_crest_factor', 'vibration_rms_kurtosis', 'vibration_rms_skewness',
    'temperature_mean', 'temperature_std',
    'pressure_mean', 'pressure_std',
    'current_mean', 'current_std'
]

# Filter available features
available = [f for f in selected_features if f in features_df.columns]

if available:
    correlation = features_df[available].corr()
    
    plt.figure(figsize=(12, 10))
    sns.heatmap(correlation, annot=True, fmt='.2f', cmap='coolwarm',
               center=0, square=True, linewidths=0.5,
               cbar_kws={"shrink": 0.8})
    plt.title('Feature Correlation Matrix', fontsize=14, fontweight='bold', pad=20)
    plt.tight_layout()
    plt.savefig('../results/figures/feature_correlation.png', dpi=300, bbox_inches='tight')
    plt.show()

# %% Cell 9: Normal vs Anomaly Feature Comparison
print("\n" + "="*80)
print("NORMAL vs ANOMALY FEATURE COMPARISON")
print("="*80)

if 'is_anomaly' in features_df.columns:
    # Select a few key features
    key_features = [
        'vibration_rms_mean', 'vibration_rms_rms', 'vibration_rms_crest_factor',
        'temperature_mean', 'pressure_mean', 'current_mean'
    ]
    
    available_key = [f for f in key_features if f in features_df.columns]
    
    if available_key:
        fig, axes = plt.subplots(2, 3, figsize=(16, 10))
        axes = axes.flatten()
        
        for i, feature in enumerate(available_key):
            ax = axes[i]
            
            # Normal distribution
            normal_data = features_df.loc[features_df['is_anomaly'] == 0, feature]
            ax.hist(normal_data, bins=30, alpha=0.6, label='Normal',
                   color='green', density=True, edgecolor='black')
            
            # Anomaly distribution
            anomaly_data = features_df.loc[features_df['is_anomaly'] == 1, feature]
            if len(anomaly_data) > 0:
                ax.hist(anomaly_data, bins=30, alpha=0.6, label='Anomaly',
                       color='red', density=True, edgecolor='black')
            
            ax.set_xlabel(feature.replace('_', ' ').title(), fontsize=9)
            ax.set_ylabel('Density', fontsize=9)
            ax.legend(fontsize=8)
            ax.grid(True, alpha=0.3)
        
        plt.suptitle('Feature Distributions: Normal vs Anomaly', fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.savefig('../results/figures/normal_vs_anomaly_features.png', dpi=300, bbox_inches='tight')
        plt.show()
        
        # Statistical comparison
        print("\nStatistical Comparison:")
        for feature in available_key[:3]:
            normal_mean = features_df.loc[features_df['is_anomaly'] == 0, feature].mean()
            anomaly_mean = features_df.loc[features_df['is_anomaly'] == 1, feature].mean()
            change = ((anomaly_mean - normal_mean) / normal_mean * 100)
            
            print(f"\n{feature}:")
            print(f"  Normal Mean: {normal_mean:.4f}")
            print(f"  Anomaly Mean: {anomaly_mean:.4f}")
            print(f"  Change: {change:+.2f}%")

# %% Cell 10: Save Features
print("\n" + "="*80)
print("SAVING FEATURES")
print("="*80)

saved_path = engineer.save_features(features_df, 'machine_001_features.csv')
print(f"✓ Features saved to: {saved_path}")

# Save statistics
stats_path = Path(config.get('paths.data_features')) / 'feature_statistics.csv'
stats_df.to_csv(stats_path, index=False)
print(f"✓ Statistics saved to: {stats_path}")

# %% Cell 11: Summary
print("\n" + "="*80)
print("FEATURE ENGINEERING SUMMARY")
print("="*80)

print(f"\n✓ Processed {len(df_sample):,} raw samples")
print(f"✓ Created {len(features_df):,} feature windows")
print(f"✓ Extracted {len(feature_cols)} features per window")
print(f"✓ Anomaly rate: {features_df['is_anomaly'].mean()*100:.2f}%")

print(f"\nFeature Categories:")
print(f"  • Basic statistics: mean, std, min, max, median, range")
print(f"  • Statistical moments: skewness, kurtosis")
print(f"  • Signal characteristics: RMS, crest factor, peak-to-peak")
print(f"  • Shape factors: impulse, clearance, shape")
print(f"  • Energy metrics: total energy, power")
print(f"  • Zero-crossing rate")
print(f"  • Peak analysis")
print(f"  • Percentiles")
print(f"  • Rolling window statistics")
print(f"  • Rate of change indicators")

print(f"\nNext Steps:")
print(f"  1. Feature selection (remove low-variance features)")
print(f"  2. Feature scaling/normalization")
print(f"  3. Model training with engineered features")
print(f"  4. Feature importance analysis")

print("\n" + "="*80)
print("✓ Feature engineering complete!")
print("="*80)